In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import OneClassSVM, SVC
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold

In [2]:
# load dataset
df = pd.read_csv(r'F:\Koding_Kopmputasi_berbasis_jaringan\sentiment amalysis dataset\movie.csv')

In [3]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [4]:
# create feature matrix and target vector
X = df['text']
y = df['label']

# create vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# create feature matrix
X_tfidf = tfidf_vectorizer.fit_transform(X)


In [5]:
# create classifiers
classifiers = [('Naive-Bayes', MultinomialNB()),
               ('Gradient Boosting', GradientBoostingClassifier()), 
               ('AdaBoost', AdaBoostClassifier()), 
               ('Extra Trees', ExtraTreesClassifier()), 
               ('Random Forest', RandomForestClassifier()),
               ('Decision Tree', DecisionTreeClassifier()),
               ('XGBoost', XGBClassifier()),
               ('CatBoost', CatBoostClassifier()),
               ('LGBM', LGBMClassifier()),
               ('OneClassSVM', OneClassSVM()),
               ('SVC', SVC())]

# define resampling methods
resampling_methods = [('SMOTETomek', SMOTETomek(random_state=0)), ('RandomUnderSampler', RandomUnderSampler(random_state=0)), ('SMOTE', SMOTE(random_state=0))]

In [6]:
# create the stratified k-fold cross-validator
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# evaluate the classifiers using cross-validation
results = []
for classifiers in classifiers:
    model_name, clf = classifiers
    print("==={}===".format(model_name))
    for resampler_name, resampler in resampling_methods:
        accuracy_scores = []
        f1_scores = []
        precision_scores = []
        recall_scores = []
        for train_index, test_index in skf.split(X_tfidf, y):
            X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
            y_train, y_test = y[train_index], y[test_index]
            X_resampled, y_resampled = resampler.fit_resample(X_train, y_train)
            clf.fit(X_resampled, y_resampled)
            y_pred = clf.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='micro')
            precision = precision_score(y_test, y_pred, average='micro')
            recall = recall_score(y_test, y_pred, average='micro')
            accuracy_scores.append(accuracy)
            f1_scores.append(f1)
            precision_scores.append(precision)
            recall_scores.append(recall)
        results.append((model_name, resampler_name, np.mean(accuracy_scores), np.mean(f1_scores), np.max(accuracy_scores), np.max(f1_scores), np.max(precision_scores), np.max(recall_scores)))

# create dataframe from results
df_results = pd.DataFrame(results, columns=['classification_model_name', 'resampling_technique_name', 'average_accuracy_score', 'average_f1_score', 'best_accuracy_score', 'best_f1_score', 'best_precision_score', 'best_recall_score'])

# print results
print(df_results)

===Naive-Bayes===
===Gradient Boosting===
===AdaBoost===
===Extra Trees===
===Random Forest===
===Decision Tree===
===XGBoost===
===CatBoost===
Learning rate set to 0.044853
0:	learn: 0.6792556	total: 853ms	remaining: 14m 12s
1:	learn: 0.6665672	total: 1.48s	remaining: 12m 18s
2:	learn: 0.6554008	total: 2.11s	remaining: 11m 41s
3:	learn: 0.6460641	total: 2.75s	remaining: 11m 24s
4:	learn: 0.6378187	total: 3.37s	remaining: 11m 10s
5:	learn: 0.6299738	total: 4s	remaining: 11m 3s
6:	learn: 0.6225740	total: 4.63s	remaining: 10m 57s
7:	learn: 0.6161767	total: 5.26s	remaining: 10m 52s
8:	learn: 0.6095076	total: 5.89s	remaining: 10m 48s
9:	learn: 0.6045590	total: 6.51s	remaining: 10m 44s
10:	learn: 0.6001731	total: 7.14s	remaining: 10m 42s
11:	learn: 0.5949586	total: 7.77s	remaining: 10m 39s
12:	learn: 0.5896197	total: 8.39s	remaining: 10m 37s
13:	learn: 0.5850654	total: 9.07s	remaining: 10m 38s
14:	learn: 0.5817621	total: 9.78s	remaining: 10m 42s
15:	learn: 0.5777173	total: 10.5s	remaining: 

152:	learn: 0.4197726	total: 1m 41s	remaining: 9m 22s
153:	learn: 0.4192816	total: 1m 42s	remaining: 9m 21s
154:	learn: 0.4187273	total: 1m 42s	remaining: 9m 20s
155:	learn: 0.4182623	total: 1m 43s	remaining: 9m 19s
156:	learn: 0.4178234	total: 1m 44s	remaining: 9m 18s
157:	learn: 0.4173362	total: 1m 44s	remaining: 9m 18s
158:	learn: 0.4168182	total: 1m 45s	remaining: 9m 17s
159:	learn: 0.4164451	total: 1m 46s	remaining: 9m 16s
160:	learn: 0.4160557	total: 1m 46s	remaining: 9m 16s
161:	learn: 0.4156310	total: 1m 47s	remaining: 9m 15s
162:	learn: 0.4151468	total: 1m 48s	remaining: 9m 14s
163:	learn: 0.4147404	total: 1m 48s	remaining: 9m 14s
164:	learn: 0.4142883	total: 1m 49s	remaining: 9m 13s
165:	learn: 0.4137951	total: 1m 50s	remaining: 9m 13s
166:	learn: 0.4133635	total: 1m 50s	remaining: 9m 12s
167:	learn: 0.4129220	total: 1m 51s	remaining: 9m 12s
168:	learn: 0.4125463	total: 1m 52s	remaining: 9m 11s
169:	learn: 0.4121437	total: 1m 52s	remaining: 9m 11s
170:	learn: 0.4116577	total:

306:	learn: 0.3624781	total: 3m 23s	remaining: 7m 39s
307:	learn: 0.3621533	total: 3m 24s	remaining: 7m 38s
308:	learn: 0.3618276	total: 3m 24s	remaining: 7m 37s
309:	learn: 0.3613966	total: 3m 25s	remaining: 7m 37s
310:	learn: 0.3611065	total: 3m 26s	remaining: 7m 36s
311:	learn: 0.3608181	total: 3m 26s	remaining: 7m 36s
312:	learn: 0.3604928	total: 3m 27s	remaining: 7m 35s
313:	learn: 0.3601896	total: 3m 28s	remaining: 7m 34s
314:	learn: 0.3598694	total: 3m 28s	remaining: 7m 34s
315:	learn: 0.3595726	total: 3m 29s	remaining: 7m 33s
316:	learn: 0.3592761	total: 3m 30s	remaining: 7m 32s
317:	learn: 0.3589882	total: 3m 30s	remaining: 7m 32s
318:	learn: 0.3586878	total: 3m 31s	remaining: 7m 31s
319:	learn: 0.3583962	total: 3m 32s	remaining: 7m 30s
320:	learn: 0.3580520	total: 3m 32s	remaining: 7m 30s
321:	learn: 0.3577491	total: 3m 33s	remaining: 7m 29s
322:	learn: 0.3574383	total: 3m 34s	remaining: 7m 28s
323:	learn: 0.3571150	total: 3m 34s	remaining: 7m 27s
324:	learn: 0.3567987	total:

459:	learn: 0.3237217	total: 5m 4s	remaining: 5m 57s
460:	learn: 0.3235179	total: 5m 4s	remaining: 5m 56s
461:	learn: 0.3233024	total: 5m 5s	remaining: 5m 55s
462:	learn: 0.3230079	total: 5m 6s	remaining: 5m 55s
463:	learn: 0.3228070	total: 5m 6s	remaining: 5m 54s
464:	learn: 0.3226043	total: 5m 7s	remaining: 5m 53s
465:	learn: 0.3223932	total: 5m 8s	remaining: 5m 53s
466:	learn: 0.3221808	total: 5m 8s	remaining: 5m 52s
467:	learn: 0.3219223	total: 5m 9s	remaining: 5m 51s
468:	learn: 0.3217576	total: 5m 10s	remaining: 5m 51s
469:	learn: 0.3215460	total: 5m 10s	remaining: 5m 50s
470:	learn: 0.3213576	total: 5m 11s	remaining: 5m 49s
471:	learn: 0.3211666	total: 5m 11s	remaining: 5m 48s
472:	learn: 0.3209641	total: 5m 12s	remaining: 5m 48s
473:	learn: 0.3207682	total: 5m 13s	remaining: 5m 47s
474:	learn: 0.3205663	total: 5m 13s	remaining: 5m 46s
475:	learn: 0.3203557	total: 5m 14s	remaining: 5m 46s
476:	learn: 0.3201140	total: 5m 15s	remaining: 5m 45s
477:	learn: 0.3199295	total: 5m 15s	r

612:	learn: 0.2958992	total: 6m 44s	remaining: 4m 15s
613:	learn: 0.2957524	total: 6m 45s	remaining: 4m 14s
614:	learn: 0.2955875	total: 6m 45s	remaining: 4m 14s
615:	learn: 0.2954430	total: 6m 46s	remaining: 4m 13s
616:	learn: 0.2952824	total: 6m 47s	remaining: 4m 12s
617:	learn: 0.2951239	total: 6m 47s	remaining: 4m 12s
618:	learn: 0.2949753	total: 6m 48s	remaining: 4m 11s
619:	learn: 0.2948310	total: 6m 49s	remaining: 4m 10s
620:	learn: 0.2946731	total: 6m 49s	remaining: 4m 10s
621:	learn: 0.2945211	total: 6m 50s	remaining: 4m 9s
622:	learn: 0.2943785	total: 6m 51s	remaining: 4m 8s
623:	learn: 0.2942443	total: 6m 51s	remaining: 4m 8s
624:	learn: 0.2941070	total: 6m 52s	remaining: 4m 7s
625:	learn: 0.2939383	total: 6m 53s	remaining: 4m 6s
626:	learn: 0.2937784	total: 6m 53s	remaining: 4m 6s
627:	learn: 0.2936116	total: 6m 54s	remaining: 4m 5s
628:	learn: 0.2934416	total: 6m 55s	remaining: 4m 4s
629:	learn: 0.2932527	total: 6m 55s	remaining: 4m 4s
630:	learn: 0.2930791	total: 6m 56s	r

...

279:	learn: 0.3735610	total: 3m 27s	remaining: 8m 54s
280:	learn: 0.3732079	total: 3m 28s	remaining: 8m 53s
281:	learn: 0.3728921	total: 3m 29s	remaining: 8m 53s
282:	learn: 0.3725670	total: 3m 30s	remaining: 8m 53s
283:	learn: 0.3722569	total: 3m 31s	remaining: 8m 53s
284:	learn: 0.3719257	total: 3m 32s	remaining: 8m 52s
285:	learn: 0.3715899	total: 3m 33s	remaining: 8m 52s
286:	learn: 0.3712498	total: 3m 34s	remaining: 8m 52s
287:	learn: 0.3709425	total: 3m 34s	remaining: 8m 51s
288:	learn: 0.3706138	total: 3m 35s	remaining: 8m 51s
289:	learn: 0.3702811	total: 3m 36s	remaining: 8m 50s
290:	learn: 0.3699088	total: 3m 37s	remaining: 8m 50s
291:	learn: 0.3695776	total: 3m 38s	remaining: 8m 50s
292:	learn: 0.3692759	total: 3m 39s	remaining: 8m 49s
293:	learn: 0.3689823	total: 3m 40s	remaining: 8m 49s
294:	learn: 0.3686987	total: 3m 41s	remaining: 8m 48s
295:	learn: 0.3683733	total: 3m 42s	remaining: 8m 48s
296:	learn: 0.3680550	total: 3m 43s	remaining: 8m 47s
297:	learn: 0.3676885	total:

432:	learn: 0.3321791	total: 5m 30s	remaining: 7m 12s
433:	learn: 0.3319155	total: 5m 31s	remaining: 7m 12s
434:	learn: 0.3316696	total: 5m 32s	remaining: 7m 11s
435:	learn: 0.3314328	total: 5m 32s	remaining: 7m 10s
436:	learn: 0.3312254	total: 5m 33s	remaining: 7m 10s
437:	learn: 0.3310281	total: 5m 34s	remaining: 7m 9s
438:	learn: 0.3308053	total: 5m 35s	remaining: 7m 8s
439:	learn: 0.3305876	total: 5m 36s	remaining: 7m 7s
440:	learn: 0.3303630	total: 5m 36s	remaining: 7m 7s
441:	learn: 0.3301388	total: 5m 37s	remaining: 7m 6s
442:	learn: 0.3299194	total: 5m 38s	remaining: 7m 5s
443:	learn: 0.3297252	total: 5m 39s	remaining: 7m 4s
444:	learn: 0.3294915	total: 5m 39s	remaining: 7m 3s
445:	learn: 0.3292999	total: 5m 40s	remaining: 7m 3s
446:	learn: 0.3290866	total: 5m 41s	remaining: 7m 2s
447:	learn: 0.3288950	total: 5m 42s	remaining: 7m 1s
448:	learn: 0.3286913	total: 5m 43s	remaining: 7m 1s
449:	learn: 0.3284312	total: 5m 43s	remaining: 7m
450:	learn: 0.3282363	total: 5m 44s	remainin

585:	learn: 0.3031235	total: 7m 30s	remaining: 5m 18s
586:	learn: 0.3029785	total: 7m 31s	remaining: 5m 17s
587:	learn: 0.3028278	total: 7m 32s	remaining: 5m 16s
588:	learn: 0.3026624	total: 7m 32s	remaining: 5m 16s
589:	learn: 0.3025082	total: 7m 33s	remaining: 5m 15s
590:	learn: 0.3023542	total: 7m 34s	remaining: 5m 14s
591:	learn: 0.3021993	total: 7m 35s	remaining: 5m 13s
592:	learn: 0.3020404	total: 7m 35s	remaining: 5m 12s
593:	learn: 0.3018791	total: 7m 36s	remaining: 5m 12s
594:	learn: 0.3017264	total: 7m 37s	remaining: 5m 11s
595:	learn: 0.3015741	total: 7m 38s	remaining: 5m 10s
596:	learn: 0.3014247	total: 7m 39s	remaining: 5m 9s
597:	learn: 0.3012581	total: 7m 39s	remaining: 5m 9s
598:	learn: 0.3011028	total: 7m 40s	remaining: 5m 8s
599:	learn: 0.3009499	total: 7m 41s	remaining: 5m 7s
600:	learn: 0.3007918	total: 7m 42s	remaining: 5m 6s
601:	learn: 0.3006274	total: 7m 43s	remaining: 5m 6s
602:	learn: 0.3004475	total: 7m 43s	remaining: 5m 5s
603:	learn: 0.3003091	total: 7m 44s

739:	learn: 0.2807532	total: 9m 30s	remaining: 3m 20s
740:	learn: 0.2806214	total: 9m 30s	remaining: 3m 19s
741:	learn: 0.2804884	total: 9m 31s	remaining: 3m 18s
742:	learn: 0.2803528	total: 9m 32s	remaining: 3m 18s
743:	learn: 0.2802685	total: 9m 33s	remaining: 3m 17s
744:	learn: 0.2801475	total: 9m 34s	remaining: 3m 16s
745:	learn: 0.2800110	total: 9m 35s	remaining: 3m 15s
746:	learn: 0.2798557	total: 9m 35s	remaining: 3m 15s
747:	learn: 0.2797111	total: 9m 36s	remaining: 3m 14s
748:	learn: 0.2795920	total: 9m 37s	remaining: 3m 13s
749:	learn: 0.2794357	total: 9m 38s	remaining: 3m 12s
750:	learn: 0.2793065	total: 9m 38s	remaining: 3m 11s
751:	learn: 0.2791741	total: 9m 39s	remaining: 3m 11s
752:	learn: 0.2790410	total: 9m 40s	remaining: 3m 10s
753:	learn: 0.2789287	total: 9m 41s	remaining: 3m 9s
754:	learn: 0.2787792	total: 9m 41s	remaining: 3m 8s
755:	learn: 0.2786494	total: 9m 42s	remaining: 3m 8s
756:	learn: 0.2785305	total: 9m 43s	remaining: 3m 7s
757:	learn: 0.2784092	total: 9m 

890:	learn: 0.2624658	total: 11m 30s	remaining: 1m 24s
891:	learn: 0.2623550	total: 11m 31s	remaining: 1m 23s
892:	learn: 0.2622345	total: 11m 32s	remaining: 1m 22s
893:	learn: 0.2621235	total: 11m 33s	remaining: 1m 22s
894:	learn: 0.2620114	total: 11m 34s	remaining: 1m 21s
895:	learn: 0.2619047	total: 11m 34s	remaining: 1m 20s
896:	learn: 0.2617789	total: 11m 35s	remaining: 1m 19s
897:	learn: 0.2616853	total: 11m 36s	remaining: 1m 19s
898:	learn: 0.2615752	total: 11m 36s	remaining: 1m 18s
899:	learn: 0.2615167	total: 11m 37s	remaining: 1m 17s
900:	learn: 0.2614004	total: 11m 38s	remaining: 1m 16s
901:	learn: 0.2613030	total: 11m 39s	remaining: 1m 15s
902:	learn: 0.2611881	total: 11m 39s	remaining: 1m 15s
903:	learn: 0.2610732	total: 11m 40s	remaining: 1m 14s
904:	learn: 0.2609403	total: 11m 41s	remaining: 1m 13s
905:	learn: 0.2608356	total: 11m 41s	remaining: 1m 12s
906:	learn: 0.2607159	total: 11m 42s	remaining: 1m 12s
907:	learn: 0.2606048	total: 11m 43s	remaining: 1m 11s
908:	learn

In [7]:
def highlight_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

# apply the formatter to the dataframe
df_results = df_results.style.apply(highlight_max, subset=pd.IndexSlice[:, ['average_accuracy_score', 'average_f1_score', 'best_accuracy_score', 'best_f1_score', 'best_precision_score', 'best_recall_score']])

# display the styled dataframe
df_results

,classification_model_name,resampling_technique_name,average_accuracy_score,average_f1_score,best_accuracy_score,best_f1_score,best_precision_score,best_recall_score
0,Naive-Bayes,SMOTETomek,0.851275,0.851275,0.854000,0.854000,0.854000,0.854000
1,Naive-Bayes,RandomUnderSampler,0.851600,0.851600,0.853875,0.853875,0.853875,0.853875
2,Naive-Bayes,SMOTE,0.851600,0.851600,0.853750,0.853750,0.853750,0.853750
3,Gradient Boosting,SMOTETomek,0.806000,0.806000,0.807625,0.807625,0.807625,0.807625
4,Gradient Boosting,RandomUnderSampler,0.806125,0.806125,0.809250,0.809250,0.809250,0.809250
5,Gradient Boosting,SMOTE,0.807100,0.807100,0.810000,0.810000,0.810000,0.810000
6,AdaBoost,SMOTETomek,0.800250,0.800250,0.804500,0.804500,0.804500,0.804500
7,AdaBoost,RandomUnderSampler,0.800550,0.800550,0.804750,0.804750,0.804750,0.804750
8,AdaBoost,SMOTE,0.800125,0.800125,0.804750,0.804750,0.804750,0.804750
9,Extra Trees,SMOTETomek,0.864300,0.864300,0.868000,0.868000,0.868000,0.868000
